In [1]:
# Importing necessary libraries
import selenium
import pandas as pd
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

## Scraping Reviews from https://www.flipkart.com/

In [2]:
# Connecting to the wedriver
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()          # Maximizing the window

In [3]:
# Getting the webpage of mentioned url
url = 'https://www.flipkart.com/'
driver.get(url)
time.sleep(2)
# Clicking on cancel button
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [4]:
# Listing dowm the items that need to be searched
srch_items = ['laptops','phones','smart watches','Monitors']

In [5]:
# Creating empty lists to fetch required data
title = []            # Title of the product
review_text = []      # text of the review or comments of the particular product
ratings = []          # Rating of the product

In [6]:
# Defining function to scrap the required data of products
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return   

In [7]:
# Creating empty list to fetch the page urls of the products
urls = []

In [8]:
for i in srch_items:
    # Find search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking on search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [9]:
# Checking the length of the url
len(urls)

288

In [10]:
# Fetching the required data from the website
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass
    
# Calling the function
    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue

In [11]:
# Checking the length of the columns
len(ratings), len(review_text), len(title)

(19140, 19140, 19140)

In [12]:
# Creating the dataframe for scraped data from the flipkart website
data = list(zip(title,review_text,ratings))
df1 = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df1

,Review_title,Reiew_text,Ratings
0,Simply awesome,It's the best I got in this segment. It has a ...,5
1,Fabulous!,i will say only 5 things\n1. cinebench r23 mul...,5
2,Terrific purchase,I get very fast delivery 🚚.\nNext its working ...,5
3,Excellent,Definitely a good laptop at its price point.\n...,5
4,Value-for-money,"Its a great laptop. I would recommend it, have...",4
...,...,...,...
19135,Simply awesome,Just awesome. Guys to get full hd or 1920*1080...,5
19136,Worst courier service from flipkart,"Product is good, But flipkart took nearly a mo...",3
19137,Awesome,Really Awesome one...go for it,5
19138,Brilliant,super,5


In [13]:
# Saving the scraped data into csv file
df1.to_csv("Flipkrt_reviews.csv")

## Scraping Reviews from https://www.amazon.in/

In [22]:
# Connecting to the webdriver
driver = webdriver.Chrome("chromedriver-Copy1.exe")
driver.maximize_window()              # Maximizing the window

In [23]:
# Getting the webpage of the mentioned url
url = 'https://www.amazon.in/'
driver.get(url)
time.sleep(2)

In [24]:
# Listing down the items to be searched
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']

In [25]:
# Creating empty lists to fetch required data
title = []           # Title of the product
review_text = []     # Text of the review or comments of the particular product
ratings = []         # Rating of the product

In [ ]:
# Fetching the required data from the website
for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  # Clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 

In [ ]:
#creating a dataframe
data = list(zip(title,review_text,ratings))
df2 = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df2

In [ ]:
# Saving the data into csv file
df2.to_csv("Amazon_reviews.csv")

In [ ]:
# lets load the flipkart reviews file
flpkrt = pd.read_csv("Flipkrt_reviews.csv")
flpkrt

In [3]:
import pandas as pd
flipkrt = pd.read_csv("Flipkrt_reviews.csv")

In [4]:
flipkrt

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Simply awesome,It's the best I got in this segment. It has a ...,5.0
1,1,Fabulous!,i will say only 5 things\n1. cinebench r23 mul...,5.0
2,2,Terrific purchase,I get very fast delivery 🚚.\nNext its working ...,5.0
3,3,Excellent,Definitely a good laptop at its price point.\n...,5.0
4,4,Value-for-money,"Its a great laptop. I would recommend it, have...",4.0
...,...,...,...,...
19135,19135,Simply awesome,Just awesome. Guys to get full hd or 1920*1080...,5.0
19136,19136,Worst courier service from flipkart,"Product is good, But flipkart took nearly a mo...",3.0
19137,19137,Awesome,Really Awesome one...go for it,5.0
19138,19138,Brilliant,super,5.0


In [5]:
amazon = pd.read_csv("Amazon_reviews.csv")
amazon

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Good laptop,\n Laptop is good for moderate users.<br>Has ...,3.0 out of 5 stars
1,1,Very bad display - not HD,\n Incredibly bad display . Not HD as describ...,1.0 out of 5 stars
2,2,Just go for it,\n Best ever laptop under 50000\n,5.0 out of 5 stars
3,3,Better multicore performance,"\n Better multi-core performance (i3, 11th ge...",5.0 out of 5 stars
4,4,NaN,NaN,NaN
...,...,...,...,...
58406,58406,Large! Warm! Comfortable!,\n Large! Warm! Comfortable and must buy for...,5.0 out of 5 stars
58407,58407,Excellent,\n Its too comfortable and excellent quality ...,5.0 out of 5 stars
58408,58408,Good one,\n Heavy but value for money. Easy to wash in...,5.0 out of 5 stars
58409,58409,Smooth and soft,\n Nice and soft\n,5.0 out of 5 stars


In [6]:
# Concatenating the two datasets 
reviews = pd.concat([amazon, flipkrt],ignore_index=True)
reviews

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Good laptop,\n Laptop is good for moderate users.<br>Has ...,3.0 out of 5 stars
1,1,Very bad display - not HD,\n Incredibly bad display . Not HD as describ...,1.0 out of 5 stars
2,2,Just go for it,\n Best ever laptop under 50000\n,5.0 out of 5 stars
3,3,Better multicore performance,"\n Better multi-core performance (i3, 11th ge...",5.0 out of 5 stars
4,4,NaN,NaN,NaN
...,...,...,...,...
77546,19135,Simply awesome,Just awesome. Guys to get full hd or 1920*1080...,5.0
77547,19136,Worst courier service from flipkart,"Product is good, But flipkart took nearly a mo...",3.0
77548,19137,Awesome,Really Awesome one...go for it,5.0
77549,19138,Brilliant,super,5.0


I have successfully scraped the required data from the websites flipcart.com and amazon.com. The dataframe consists of 77551 rows and 3 columns. And I have saved this dataframe in both excel and csv formats.

In [7]:
# Concatenating the two datasets 
reviews = pd.concat([amazon, flipkrt],ignore_index=True)
reviews

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Good laptop,\n Laptop is good for moderate users.<br>Has ...,3.0 out of 5 stars
1,1,Very bad display - not HD,\n Incredibly bad display . Not HD as describ...,1.0 out of 5 stars
2,2,Just go for it,\n Best ever laptop under 50000\n,5.0 out of 5 stars
3,3,Better multicore performance,"\n Better multi-core performance (i3, 11th ge...",5.0 out of 5 stars
4,4,NaN,NaN,NaN
...,...,...,...,...
77546,19135,Simply awesome,Just awesome. Guys to get full hd or 1920*1080...,5.0
77547,19136,Worst courier service from flipkart,"Product is good, But flipkart took nearly a mo...",3.0
77548,19137,Awesome,Really Awesome one...go for it,5.0
77549,19138,Brilliant,super,5.0


In [8]:
# Dropping unwanted column
reviews.drop(columns = 'Unnamed: 0', inplace=True)

In [9]:
# Displaying the merged dataset
reviews

,Review_title,Reiew_text,Ratings
0,Good laptop,\n Laptop is good for moderate users.<br>Has ...,3.0 out of 5 stars
1,Very bad display - not HD,\n Incredibly bad display . Not HD as describ...,1.0 out of 5 stars
2,Just go for it,\n Best ever laptop under 50000\n,5.0 out of 5 stars
3,Better multicore performance,"\n Better multi-core performance (i3, 11th ge...",5.0 out of 5 stars
4,NaN,NaN,NaN
...,...,...,...
77546,Simply awesome,Just awesome. Guys to get full hd or 1920*1080...,5.0
77547,Worst courier service from flipkart,"Product is good, But flipkart took nearly a mo...",3.0
77548,Awesome,Really Awesome one...go for it,5.0
77549,Brilliant,super,5.0


In [11]:
# Saving the data into csv file
reviews.to_csv("Review_Rating.csv",index=False)

In [12]:
# Saving the data into excel file
reviews.to_excel("Review_Ratings.xlsx",index=False)

### DONE